In [124]:
import geopandas as gpd
import folium
import rioxarray as riox
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [125]:
neighborhoods_frankfurt = gpd.read_file(r"C:\Users\nb\OneDrive\Documents\Halle\Hackathon local data\1 Land Prices\Land_Prices_Neighborhood_Frankfurt_am_Main.gpkg")
buildings_frankfurt= gpd.read_file(r"C:\Users\nb\OneDrive\Documents\Halle\Hackathon local data\4 Buildings\Buildings_Frankfurt_am_Main.gpkg")

In [126]:
neighborhoods_frankfurt.head()

,Neighborhood_FID,Land_Value,Area_Types,Area_Count,City_Name,geometry
0,1,6998.5,MixedUseArea_Residential,7,Frankfurt am Main,"MULTIPOLYGON (((4226384.447 3000227.322, 42263..."
1,2,11756.9,MixedUseArea_Residential_SpecialUseArea,21,Frankfurt am Main,"MULTIPOLYGON (((4227082.463 3001711.312, 42272..."
2,3,5312.2,MixedUseArea_Residential,9,Frankfurt am Main,"MULTIPOLYGON (((4225284.051 3000486.630, 42252..."
3,4,9779.6,CommercialBuildingArea_MixedUseArea_Residentia...,20,Frankfurt am Main,"MULTIPOLYGON (((4225219.883 3002076.156, 42252..."
4,5,4236.2,MixedUseArea_OtherArea_Residential_SpecialUseArea,20,Frankfurt am Main,"MULTIPOLYGON (((4225993.384 3003603.082, 42260..."


In [127]:
buildings_frankfurt.head()

,Build_ID,Building_TypeGen,Building_Type,Building_Height,Building_Age,Neighborhood_FID,Neighborhood_Name,City_Name,geometry
0,1,non-residential,commercial,27.5,2011.0,15,Flughafen,Frankfurt am Main,"MULTIPOLYGON (((4218155.247 2994285.747, 42181..."
1,2,residential,apartments,NaN,NaN,6,Nordend-West,Frankfurt am Main,"MULTIPOLYGON (((4226201.801 3002499.140, 42262..."
2,3,non-residential,office,10.0,NaN,24,Niederursel,Frankfurt am Main,"MULTIPOLYGON (((4223711.904 3007209.140, 42237..."
3,4,non-residential,sports_centre,NaN,NaN,9,Bornheim,Frankfurt am Main,"MULTIPOLYGON (((4229529.535 3002303.265, 42296..."
4,5,unknown,no,NaN,NaN,28,Eckenheim,Frankfurt am Main,"MULTIPOLYGON (((4227175.409 3004644.535, 42272..."


In [128]:
buildings_frankfurt['Building_Type'].unique()

array(['commercial', 'apartments', 'office', 'sports_centre', 'no', None,
       'stadium', 'garages', 'cathedral', 'church', 'public',
       'transportation', 'industrial', 'crypts', 'retail', 'school',
       'parking', 'civic', 'kindergarten', 'roof', 'university', 'hotel',
       'government', 'dormitory', 'hospital', 'supermarket',
       'construction', 'warehouse', 'train_station', 'service',
       'detached', 'terrace', 'residential', 'garage', 'shed', 'house',
       'religious', 'semidetached_house', 'bungalow', 'carport', 'kiosk',
       'bunker', 'greenhouse', 'watchtower', 'farm_auxiliary',
       'prefabricated', 'college', 'fire_station', 'hangar', 'stable',
       'mosque', 'container', 'sports_hall', 'ruins', 'tent',
       'data_center', 'retail;warehouse', 'bridge', 'storage_tank', 'hut',
       'toilets', 'water_tower', 'museum', 'cinema', 'grandstand',
       'chapel', 'farm', 'nursing_home', 'pagoda', 'Clubhouse',
       'transformer_tower', 'funeral_hall', 'boa

In [129]:
buildings_frankfurt.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 117117 entries, 0 to 117116
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   Build_ID           117117 non-null  int64   
 1   Building_TypeGen   46240 non-null   object  
 2   Building_Type      46300 non-null   object  
 3   Building_Height    28493 non-null   float64 
 4   Building_Age       254 non-null     float64 
 5   Neighborhood_FID   117117 non-null  int64   
 6   Neighborhood_Name  117117 non-null  object  
 7   City_Name          117117 non-null  object  
 8   geometry           117117 non-null  geometry
dtypes: float64(2), geometry(1), int64(2), object(4)
memory usage: 8.0+ MB


In [130]:
buildings_frankfurt['Building_Height'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 117117 entries, 0 to 117116
Series name: Building_Height
Non-Null Count  Dtype  
--------------  -----  
28493 non-null  float64
dtypes: float64(1)
memory usage: 915.1 KB


## I construct a dataframe with a row for each neighborhood. 

### first: average building height
problem: lot's of missings

In [137]:
neighb_aggr = buildings_frankfurt.pivot_table(values = 'Building_Height', index = 'Neighborhood_FID', columns = 'Build_ID')

In [138]:
neighb_aggr['avg_height']=neighb_aggr.sum(axis=1)

In [139]:
neighb_aggr_final = pd.DataFrame()

In [140]:
neighb_aggr_final['avg_height'] = neighb_aggr['avg_height']

In [141]:
neighb_aggr_final.head()

,avg_height
Neighborhood_FID,
1,954.5
2,2907.1
3,1693.5
4,4654.8
5,5052.0


### second: number of buildings

create column which is =1 if residual building and =0 if non-residual building (column name: resid) and the other way around (nresid)

In [142]:
buildings_frankfurt['resid']= np.where(
    buildings_frankfurt['Building_TypeGen'] == 'residential', 1, np.where(
    buildings_frankfurt['Building_TypeGen'] == 'non-residential' , 0, 0)) 

In [143]:
buildings_frankfurt['nresid']= np.where(
    buildings_frankfurt['Building_TypeGen'] == 'non-residential', 1, np.where(
    buildings_frankfurt['Building_TypeGen'] == 'residential' , 0, 0)) 

In [144]:
buildings_frankfurt[['Building_TypeGen', 'resid', 'nresid']]

,Building_TypeGen,resid,nresid
0,non-residential,0,1
1,residential,1,0
2,non-residential,0,1
3,non-residential,0,1
4,unknown,0,0
...,...,...,...
117112,None,0,0
117113,None,0,0
117114,None,0,0
117115,None,0,0


In [147]:
neighb_aggr2 = buildings_frankfurt.pivot_table(values = 'resid', index = 'Neighborhood_FID', columns = 'Build_ID')

In [148]:
neighb_aggr2['resid']=neighb_aggr2.sum(axis=1)

In [149]:
neighb_aggr_final['resid'] = neighb_aggr2['resid']

In [150]:
neighb_aggr_final.head()

,avg_height,resid
Neighborhood_FID,,
1,954.5,57.0
2,2907.1,68.0
3,1693.5,52.0
4,4654.8,276.0
5,5052.0,493.0


In [158]:
neighb_aggr = buildings_frankfurt.pivot_table(values = 'nresid', index = 'Neighborhood_FID', columns = 'Build_ID')

In [153]:
neighb_aggr['nresid']=neighb_aggr.sum(axis=1)

In [154]:
neighb_aggr_final['nresid'] = neighb_aggr['nresid']

In [157]:
neighb_aggr_final.head()

,avg_height,resid,nresid,build_count
Neighborhood_FID,,,,
1,954.5,57.0,15.0,57.0
2,2907.1,68.0,126.0,68.0
3,1693.5,52.0,78.0,52.0
4,4654.8,276.0,204.0,276.0
5,5052.0,493.0,154.0,493.0


I suspect that some of the buildings in the dataset are not really buildings but subway entrances e.g. To check that I look at the type of buildings of buildings indicated as residential and non-residential

In [160]:
buildings_frankfurt[buildings_frankfurt['Building_TypeGen']== 'residential']['Building_Type'].unique()

array(['apartments', 'dormitory', 'detached', 'residential', 'house',
       'semidetached_house', 'bungalow', 'boathouse', 'semi', 'houseboat',
       'cabin'], dtype=object)

In [161]:
buildings_frankfurt[buildings_frankfurt['Building_TypeGen']== 'non-residential']['Building_Type'].unique()

array(['commercial', 'office', 'sports_centre', 'stadium', 'garages',
       'church', 'public', 'transportation', 'industrial', 'retail',
       'school', 'parking', 'civic', 'kindergarten', 'roof', 'university',
       'hotel', 'government', 'hospital', 'supermarket', 'construction',
       'warehouse', 'train_station', 'service', 'terrace', 'garage',
       'shed', 'carport', 'kiosk', 'bunker', 'greenhouse',
       'farm_auxiliary', 'college', 'fire_station', 'hangar', 'stable',
       'mosque', 'container', 'sports_hall', 'ruins', 'tent',
       'storage_tank', 'hut', 'toilets', 'grandstand', 'chapel', 'farm',
       'transformer_tower', 'barn', 'shelter', 'static_caravan', 'temple',
       'substation', 'shop'], dtype=object)

Conclusion: residential are really only residential but non-residential is everthing else. 

In [162]:
neighb_aggr_final['resid_count']= neighb_aggr_final.groupby('Neighborhood_FID')['resid'].sum()

In [163]:
neighb_aggr_final['nresid_count']= neighb_aggr_final.groupby('Neighborhood_FID')['nresid'].sum()

In [164]:
neighb_aggr_final['build_count']= neighb_aggr_final['resid_count'] + neighb_aggr_final['nresid_count']

In [166]:
neighb_aggr_final['resid_share']= neighb_aggr_final['resid_count'] / neighb_aggr_final['build_count']

In [167]:
neighb_aggr_final.head()

,avg_height,resid,nresid,build_count,resid_count,nresid_count,resid_share
Neighborhood_FID,,,,,,,
1,954.5,57.0,15.0,72.0,57.0,15.0,0.791667
2,2907.1,68.0,126.0,194.0,68.0,126.0,0.350515
3,1693.5,52.0,78.0,130.0,52.0,78.0,0.400000
4,4654.8,276.0,204.0,480.0,276.0,204.0,0.575000
5,5052.0,493.0,154.0,647.0,493.0,154.0,0.761978


In [168]:
buildings_frankfurt.head()

,Build_ID,Building_TypeGen,Building_Type,Building_Height,Building_Age,Neighborhood_FID,Neighborhood_Name,City_Name,geometry,resid,nresid
0,1,non-residential,commercial,27.5,2011.0,15,Flughafen,Frankfurt am Main,"MULTIPOLYGON (((4218155.247 2994285.747, 42181...",0,1
1,2,residential,apartments,NaN,NaN,6,Nordend-West,Frankfurt am Main,"MULTIPOLYGON (((4226201.801 3002499.140, 42262...",1,0
2,3,non-residential,office,10.0,NaN,24,Niederursel,Frankfurt am Main,"MULTIPOLYGON (((4223711.904 3007209.140, 42237...",0,1
3,4,non-residential,sports_centre,NaN,NaN,9,Bornheim,Frankfurt am Main,"MULTIPOLYGON (((4229529.535 3002303.265, 42296...",0,1
4,5,unknown,no,NaN,NaN,28,Eckenheim,Frankfurt am Main,"MULTIPOLYGON (((4227175.409 3004644.535, 42272...",0,0


also interesting: the share of houses vs. apartments - number of people per square meter

In [217]:
neighb_aggr = pd.DataFrame()
neighb_aggr[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']] = buildings_frankfurt[buildings_frankfurt['Building_Type']=='house'].groupby('Neighborhood_FID').count()
neighb_aggr_final['house_count']= neighb_aggr['1']
neighb_aggr_final.head()

,avg_height,resid,nresid,build_count,resid_count,nresid_count,resid_share,house_count
Neighborhood_FID,,,,,,,,
1,954.5,57.0,15.0,72.0,57.0,15.0,0.791667,4.0
2,2907.1,68.0,126.0,194.0,68.0,126.0,0.350515,3.0
3,1693.5,52.0,78.0,130.0,52.0,78.0,0.400000,1.0
4,4654.8,276.0,204.0,480.0,276.0,204.0,0.575000,1.0
5,5052.0,493.0,154.0,647.0,493.0,154.0,0.761978,17.0


In [219]:
neighb_aggr = pd.DataFrame()
neighb_aggr[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']] = buildings_frankfurt[buildings_frankfurt['Building_Type']=='apartment'].groupby('Neighborhood_FID').count()
neighb_aggr_final['apartment_count']= neighb_aggr['1']
neighb_aggr_final

,avg_height,resid,nresid,build_count,resid_count,nresid_count,resid_share,house_count,apartment_count
Neighborhood_FID,,,,,,,,,
1,954.5,57.0,15.0,72.0,57.0,15.0,0.791667,4.0,NaN
2,2907.1,68.0,126.0,194.0,68.0,126.0,0.350515,3.0,NaN
3,1693.5,52.0,78.0,130.0,52.0,78.0,0.400000,1.0,NaN
4,4654.8,276.0,204.0,480.0,276.0,204.0,0.575000,1.0,NaN
5,5052.0,493.0,154.0,647.0,493.0,154.0,0.761978,17.0,NaN
6,7173.5,1085.0,234.0,1319.0,1085.0,234.0,0.822593,11.0,NaN
7,3291.5,1275.0,128.0,1403.0,1275.0,128.0,0.908767,143.0,NaN
8,10443.5,1546.0,617.0,2163.0,1546.0,617.0,0.714748,38.0,NaN
9,16737.5,2397.0,572.0,2969.0,2397.0,572.0,0.807343,362.0,NaN
